<h1>CBC Padding Oracle Attack</h1>

v1.0: Ryan Lehmkuhl, Ben Hoberman

v2.0: Ben Hoberman

Run the following block to install packages needed for this notebook:

In [1]:
%pip install cryptography flask requests

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 94 kB 1.6 MB/s eta 0:00:011
     |████████████████████████████████| 298 kB 3.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import base64
import os
from IPython.display import clear_output
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher
from cryptography.hazmat.primitives.ciphers.algorithms import AES
from cryptography.hazmat.primitives.ciphers.modes import ECB
from requests.adapters import HTTPAdapter
from requests import Session
from tests import test1, test2, test3, test4, test5, test6, test7, test8
from helpers import PKCS7_pad, PKCS7_unpad, valid_pad, permute
%load_ext autoreload
%autoreload 2

# Question 1: Introduction

Trying to implement crypto schemes yourself can be very dangerous. In the words of Runa Sandvi, The New York Times senior director of information security, "Asking why you should not roll your own crypto is a bit like asking why you should not design your own aircraft engine."

While many of the concepts we've reviewed in class may seem intuitive and simple, even the subtlest leakage of information can completely compromise any hope for confidentiality.

We will demonstrate this by completely decrypting a message encrypted with AES-CBC using a **padding oracle attack**.


## Padding

First off, what is padding and why do we need it? Recall that a block cipher such as AES encrypts messages of a *fixed size*. When the plaintext is larger than the blocksize, we use a block chaining mode, such as CBC. However, what do we do when the message isn't a multiple of the block size? For this, we have to pad the message. 

Padding is simply adding some form of junk to the end of a message in order to fit it to the block size. For example, suppose our our plaintext was the following string:
```
Message   |N  e   e  d  _  p  a   d  d   i  n  g  !   \0 ?  ? 
Index     |0  1   2  3  4  5  6   7  8   9  10 11 12  13 14 15
Hex       |4e 65  65 64 20 70 61  64 64  69 6e 67 21  00 ?? ??
```

Providing this as input to AES with blocksize of 16 bytes will throw an error. Instead we need to replace the suffix of the message (represented with question marks) with some form of padding. Note that we must choose our padding such that removing it doesn't *accidently change our message*. For instance, if we chose to simply append zeroes, we would erase the null terminator of the string during unpadding.

The most common padding method for symettric ciphers is PKCS#7 (PKCS stands for Public Key Cryptography Standard). It works by appending the number of padding bytes to the end of the message. So our message from before would become:
```
Message   |N  e   e  d  _  p  a   d  d   i  n  g  !   \0 0x02  0x02 
Index     |0  1   2  3  4  5  6   7  8   9  10 11 12  13 14    15
Hex       |4e 65  65 64 20 70 61  64 64  69 6e 67 21  00 02    02
```

Note that in if the message is a multiple of the block size, another block of all 16s (```0x10```) is added.

**Fill-in and run the following block to get a feel for how this works.**

In [3]:
### TODO: Type in a message here to see how it's padded correctly
msg = b"oski is terrifying"

# Pads the message and splits it into blocks
padded_msg = PKCS7_pad(msg)
plaintext = [msg[i:i + 16].decode() for i in range(0, len(msg), 16)]
padded_plaintext = [padded_msg[i:i + 16] for i in range(0, len(padded_msg), 16)]

# Prints everything - decodes the blocks of byte objects to strings
print(f"Plaintext w/o padding: {plaintext}")
print(f"Length: {len(msg)}")
print(f"Message with padding: {[x.decode() for x in padded_plaintext]}")
print(f"Length: {len(padded_msg)}\n")
print(f"Hex of padded message:")
for i in range(len(padded_plaintext)):
    print(f"Block {i}: ", " ".join('0x{:02x}'.format(c) for c in padded_plaintext[i]))

Plaintext w/o padding: ['oski is terrifyi', 'ng']
Length: 18
Message with padding: ['oski is terrifyi', 'ng\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e']
Length: 32

Hex of padded message:
Block 0:  0x6f 0x73 0x6b 0x69 0x20 0x69 0x73 0x20 0x74 0x65 0x72 0x72 0x69 0x66 0x79 0x69
Block 1:  0x6e 0x67 0x0e 0x0e 0x0e 0x0e 0x0e 0x0e 0x0e 0x0e 0x0e 0x0e 0x0e 0x0e 0x0e 0x0e


## Padding Oracles
We now know why padding is important and how to implement it, but how can a faulty implementation break our crypto system? For this, we introduce the concept of a **padding oracle**.

In cryptography, an oracle is a queryable 'black box' (a function with unknown inner-workings) which provides some piece of information which otherwise would not be available. For example, when we studied the IND-CPA game, the challenger acted as an **encryption oracle** since the adversary could query it on a given message and receive back a ciphertext without knowing how the encryption was done (ie. which key was used). 

A padding oracle takes as input, some ciphertext `c`, and returns `True` if the (decrypted) ciphertext is properly padded and `False` otherwise. We've defined a function, `valid_pad` which acts as such a padding oracle for PKCS7.

For the following two parts, you don't need to worry at all about a padding oracle or any cryptography &mdash; you're just working with plaintext, and the point is for you to get just a little bit of a sense for padding in general.

## 1.1

**Below, input a last byte for `padded_msg` which will cause `valid_pad` to return `False`.**

In [6]:
msg = b"oski is terrifying" # do not change this

### TODO: Choose a last byte (written as a number) that makes the padding invalid
invalid_last_byte = 12 ### YOUR CODE HERE ###

# Autograder
test1(invalid_last_byte)

All tests passed!


## 1.2

**Next, give *two* different last bytes for `padded_msg` which will cause `valid_pad` to return `True`.**

In [7]:
msg = b"oski is terrifying" # do not change this

### TODO: Choose two different last bytes (written as a number) that makes the padding valid
valid_last_byte_1 = 1 ### YOUR CODE HERE ###
valid_last_byte_2 = 14 ### YOUR CODE HERE ###

# Autograder
test2(valid_last_byte_1, valid_last_byte_2)

All tests passed!


Many real systems naturally act as padding oracles. Consider a web server which uses AES-CBC to encrypt communications between it's clients (early versions of TLS did this). If a client sends a message with invalid padding, the exception might cause the web server to respond with something like:
<center>
<img src="https://i.imgur.com/IFhVUbJ.png" align="center" style="height:400px" />
</center>

Why is this bad? At a strictly fundamental level, the resulting error leaks information about the plaintext which should never be allowed in any cryptographical system. But come on... detecting incorrect padding can't be that bad? Right? **WRONG.** This simple leakage ends up completely destroying any hope for confidentiality with the encryption scheme. To see why let's review how the CBC block chaining mode works.

## CBC
Decryption in CBC mode is as follows:
<center>
<img src="https://i.imgur.com/CRUh4nu.png" align="center" style="height:300px" />
</center>

In particular, we are interested in the decryption of a single block - especially the temporary block state that occurs before the XOR:
<center>
<img src="https://i.imgur.com/ufUzmaH.png" align="center" style="height:250px" />
</center>

# Question 2: Formulating the Attack

Using the previous image as a reference, denote the block cipher decryption function as $D(\cdot)$, the last ciphertext block as $C_{n}$, the preceding ciphertext block as $C_{n-1}$, the resulting plaintext block as $P_n$, and the last temporary state block as $T_n$.

## 2.1

**Express $P_n$ and $T_n$ in terms of $D$, $C_n$, and $C_{n-1}$.**

*Note: `\oplus` lets you write $\oplus$ in LaTeX*

**<span style = "color: red">(YOUR ANSWER HERE)</span>**

## 2.2

**Express $P_n$ in terms of $C_{n-1}$ and $T_n$.**

**<span style = "color: red">(YOUR ANSWER HERE)</span>**

## 2.3

Now, we're going to start building towards a full-fledged attack based on this decryption process.

**First, implement the expression you found for $P_n$ in the code block below.**

In [ ]:
from helpers import xor_block
# xor_block(block1, block2) automatically xors two blocks of bytes together

def P_from_DCC(D, C, C_prev):
    """
    Compute P_n from D(.), C_n, and C_{n-1}.

    D:         a block cipher decryption function.
    C:         C_n, a 16-byte block of text
    C_prev:    C_{n-1}, the 16-byte ciphertext preceding C
    """
    T = ... ### YOUR CODE HERE ###
    P = ... ### YOUR CODE HERE ###
    return P

# Autograder
test3(P_from_DCC)

Now, assume that you have intercepted some ciphertext $(IV, C_1, C_2, \ldots, C_n)$ and have access to a padding oracle. You have complete freedom with what you send the padding oracle (ie. a subset of the ciphertext blocks or something completely different). Whatever you send, the padding oracle will decrypt it **using the original symmetric key** and truthfully report whether it is padded correctly.

For now, ignore all of the blocks except for $C_n$. Let's say that we send the padding oracle the ciphertext $(C_{n-1}', C_n)$ where $C_{n-1}'$ is some carefully-chosen ciphertext. Decrypting this input will result in some $P_n'$ which can be expressed in terms of $C_{n-1}'$ and $T_n$ (as shown above). Our goal is to craft our $C_{n-1}'$ s.t. $P_n'$ has valid padding no matter what.

## 2.4.

**In terms of $T_n[15]$, what must $C_{n-1}'[15]$ be to *guarantee* a correctly padded $P_n'$?**

**<span style = "color: red">(YOUR ANSWER HERE)</span>**

## 2.5

**Implement your answer from part 2.4 in the following function to test it out.**

In [ ]:
def pad_correctly(T_byte):
    """
    Computes C'_{n-1}[15] given T_n[15] which results
    in a correctly padded P_n[15].
    
    T_byte: T_n[15]
    """
    # Hint: ^ is XOR in Python.
    return ... ### YOUR CODE HERE ###

# Autograder
test4(pad_correctly)

## 2.6.

Your answer to part 1.4 guarantees that the ciphertext is padded correctly regardless of the rest of $P_n'$, but it is also sometimes possible for a *different* value of $C'_{n-1}[15]$ to result in valid padding (depending on the rest of $P_n'$).

**When does this different value result in valid padding?**

**<span style = "color: red">(YOUR ANSWER HERE)</span>**

## 2.7

**Test your answer by filling out the function below.**

Note that you will need to use your knowledge of the temporary state, which is not normally available.

In [ ]:
def pad_correctly_alt(T_byte, P_byte):
    """
    Computes C'_{n-1}[15] given T_n[15] and P_n[15] which results
    in a correctly padded P_n[15]. Should return a different byte
    than your last answer!
    
    T_byte: T_n[15]
    P_byte: P_n[15] - the last byte of the real message.
    """
    return ... ### YOUR CODE HERE ###

# Autograder
test5(pad_correctly_alt)

## 2.8

**Using what you've discovered so far, how might you leverage the padding oracle to learn these one or two possible values for $C_{n-1}'[15]$?**

*Hint: a brute force attack that only needs to try a small number of options is very fast. 256 is very small.*

**<span style = "color: red">(YOUR ANSWER HERE)</span>**

## 2.9

If a particular byte has two potential values with correct padding, we need to figure out which one is which. In the function below, you're given the byte which *always* results in correct padding.

**Fill in the function to calculate the corresponding byte of the plaintext.**

In [ ]:
def recover_byte(C_byte, C_prime_byte):
    """
    Given the modified ciphertext C'_{n-1}[15] which resulted in
    the padding always being correct, and the original ciphertext
    C_{n-1}[15], return the last byte of the original plaintext P_n[15].
    
    C_byte: C_{n-1}[15] - the original last byte of the ciphertext
    C_prime_byte: C'_{n-1}[15] - the modified last byte which resulted in correct padding all of the time
    """
    return ... ### YOUR CODE HERE ###

# Autograder --- make sure you've completed part 4 first! This test requires your implementation of pad_correctly.
test6(recover_byte, pad_correctly)

## 2.10

When running the attack you described in part 2.8, you will run into trouble if you have two candidates for a particular byte. If you got two possible values for $C_n'[15]$ that result in valid padding, **how can we modify $C_n'[14]$ to check which of these values is correct?** (In real life we won't know the message so we can't do what we did before)

*Hint: There are over 200 values that work here*

**<span style = "color: red">(YOUR ANSWER HERE)</span>**

## 2.11

**Fill in the function so that it always correctly finds the last byte of plaintext based on the last two blocks of ciphertext and a padding oracle.**

In [ ]:
def solve_last_byte(C_prev, C, oracle):
    """
    Returns the correct last byte of the original plaintext
    P_n[15] given the previous the ciphertexts C_n, C_{n-1},
    and a padding oracle
    
    C: current block (the one whose plaintext we're solving for) of the ciphertext (C_n)
    C_prev: previous block of the cipher text (C_{n-1})
    oracle: a function which returns whether (C_prev, C) are padded correctly
    """
    for byte in ...: ### YOUR CODE HERE. What are the possible values for byte? ###
        C_prev_prime = bytearray(C_prev)
        C_prev_prime[15] = ... ### YOUR CODE HERE ###
        if oracle(C_prev_prime, C):
            C_prev_prime[14] = ... ### YOUR CODE HERE ###
            if oracle(C_prev_prime, C):
                return recover_byte(C_prev[15], C_prev_prime[15])
            
# Autograder
test7(solve_last_byte)

## 2.12

**How could you extend the previous few steps to decode an entire block? An entire message?**

**<span style = "color: red">(YOUR ANSWER HERE)</span>**

## 2.13

Now, we can finally implement the full decryption.

**Fill out the function below based on prior parts in order to successfully decrypt any CBC ciphertext block based on a padding oracle.**

Remember that as you discover more of the message, the "correct" padding you need to enforce will change.

In [ ]:
def decrypt_block(C_prev, C, oracle, display = False, tail = None):
    """
    Recover plaintext P_n given ciphertext C_n, C_{n-1}, and a padding oracle.
    Don't worry about the display or tail arguments -- they're for cool visualization later.

    C: current block (the one we're solving for) of the ciphertext (C_i)
    C_prev: last block of the cipher text (C_{i-1})
    oracle: a function which returns whether (C_prev, C) are padded correctly
    """
    correct_block = bytearray(16) # Reconstructed plaintext (P from previous parts)
    temp_block = bytearray(16) # Reconstructed temporary state (T from previous parts)
    
    # Iterate over the block from end to beginning
    for i in reversed(range(16)):
        # Set the padding byte that the known bytes should be to guarantee correct padding
        padding_byte = ... ### YOUR CODE HERE. ###
        
        for byte in ...: ### YOUR CODE HERE ###
            C_prev_prime = bytearray(C_prev)
            C_prev_prime[i+1:] = xor_block(..., [padding_byte]*padding_byte) ### YOUR CODE HERE. 
                ### How do we set the already-known plaintext to a value of our choice? ###
            
            C_prev_prime[i] = ... ### YOUR CODE HERE ###
            if oracle(C_prev_prime, C):
                if i == 15: # Recall from previous parts that the last byte can have two possible values.
                    C_prev_prime[i-1] = ... ### YOUR CODE HERE ###
                    if not oracle(C_prev_prime, C):
                        continue
                        
                # We can now deduce the ith byte.
                # As you pad farther and farther back, you'll need to slightly tweak how you recover
                # the correct byte. Think back to how you derived it for the last byte; the process
                # will be very similar
                correct_block[i] = ... ### YOUR CODE HERE ###
                temp_block[i] = ... ### YOUR CODE HERE ###
                
                # Visualization code -- don't worry about it
                if display:
                    progress = correct_block.replace(b'\x00', b' ')
                    clear_output(wait=True)
                    if tail is not None:
                        progress = progress + tail
                    print(f'Recovered so far: {progress}')
                    
                break
    return correct_block

# Autograder
test8(decrypt_block)

# Question 3: Implementing the Attack
Alice has connected her home IoT devices to a central web application. This allows her to control every aspect of her home from anywhere with an Internet connection. Alice read somewhere that she should encrypt her communications non-deterministically in order to negate the threat of replay attacks, so she opted to use AES-CBC + PKCS#7 padding with a shared symmetric key.

You (Eve), discover the exposed API to Alice's web application and notice that the application caches the most recently received command. You also find that the web application will return a 500 error if sent an invalid command. Using the web application's API, can you decrypt the cached ciphertext and discover Alice's command?

## Setup

We have provided a few things for you:
* Alice's web application
* A `Client` class to interface with the web application

**If you're running the notebook locally, open a new terminal window and run:**
```
python iot.py
```
**If you're running the notebook on DataHub/Collab, execute the block below:**

In [ ]:
from iot import app
import threading
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':12000}).start() 

**To define the `Client` class, run the block below:**

In [ ]:
LOCAL_URL = 'http://127.0.0.1:12000/api/'

class Client:
    def __init__(self):
        # Start a new HTTP Session
        self.session = Session()
        self.session.mount('http://', HTTPAdapter())

        # Get the cached command
        url = LOCAL_URL + 'cache'
        response = self.session.get(url)
        content = response.json()
        if response.status_code in [400, 401, 403]:
            print('/cache API Error: ' + str(status_code))
        elif response.status_code != 200:
            print('/cache API Error: ' + str(status_code))

        # Decode the ciphertext
        self.iv = base64.b64decode(bytes(content['iv'], 'utf8'))
        self.ciphertext = base64.b64decode(bytes(content['ciphertext'], 'utf8'))

    def execute(self, iv, ciphertext):
        '''Sends ciphertext to web application. Return True if command is
        executed, False if the application returns an error'''
        if type(iv) == type(bytearray()):
            iv = bytes(iv)
        if type(ciphertext) == type(bytearray()):
            ciphertext = bytes(ciphertext)
        data = {
            'iv': base64.b64encode(iv), 
            'ciphertext': base64.b64encode(ciphertext)
            }
        response = self.session.post(LOCAL_URL + 'execute',data=data)
        status_code = response.status_code
        content = response.json()
        return content['success']

Now, we can apply our attack! **First, we need to formulate our oracle in terms of the `client`. To do so, run the code block below.**

In [ ]:
def make_network_oracle(client):
    def oracle(C_last, C):
        return client.execute(C_last, C)
    return oracle

## 3.1

Now we can finally put it all together and execute our attack against a real server. We can directly use your work from previous parts to fill out the `decrypt` function below!

In [ ]:
# No code you have to write in here :)
def decrypt(client):
    # Make the ciphertext mutable by casting to bytearray instead of bytes
    iv = bytearray(client.iv)
    ciphertext = bytearray(client.ciphertext)
    
    # Split the ciphertext into blocks
    blocks = [iv] + [ciphertext[i:i + 16] for i in range(0, len(ciphertext), 16)]
    # This stores our recovered plaintext
    plaintext = [bytearray(16) for _ in range(len(blocks))]
    oracle = make_network_oracle(client)

    # Recover each block and byte in reverse order
    for i in reversed(range(1, len(blocks))):
        C_last = blocks[i - 1]
        C = blocks[i]
        plaintext[i] = decrypt_block(C_last, C, oracle, True, b''.join(plaintext[i+1:]))
    
    return b''.join(plaintext[1:]).decode()

Run your attack:

In [ ]:
print(f"\n\nThe command is: {decrypt(Client())}")

What is the decrypted message?

**<span style = "color: red">(YOUR ANSWER HERE)</span>**

# Question 4: History and Defenses

The padding oracle that you've developed in this lab has been used in plenty of real-world attacks. The attack was first discovered (publicly) in 2002. Since 2002 is well within the era of the internet, you can find the paper published by the exploit's original authors [here](https://www.iacr.org/cryptodb/archive/2002/EUROCRYPT/2850/2850.pdf). Exploits were quickly engineered against all sorts of servers to great success.

Fixes for this original attack "fixed" the vulnerability by simply stopping the server from directly telling users whether a message was padded correctly or not. But later attacks, in particular [lucky thirteen](https://arstechnica.com/information-technology/2013/02/lucky-thirteen-attack-snarfs-cookies-protected-by-ssl-encryption/), utilized the fact that a *timing* side-channel could let an attacker statistically infer when a padding was set correctly even without the server telling them explicitly. This illustrates how hard it is to avoid side-channel attacks and why we always prefer to use schemes which fail as safely as possible.

For the most part, CBC padding oracle attacks have been patched, but there are notable circumstances in which they can still be used to great success. [Some attacks](https://en.wikipedia.org/wiki/POODLE) use the fact that users can request to use older versions of protocols for backwards compatibilty, which can directly enable old attacks to come back to life. And since implementing cryptography is *hard*, patches and fixes can result in the resurrection of previously-patched vulnerabilities. Check out [this](https://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2016-2107) CVE, an ironic example of how a patch for lucky thirteen actually *enabled* a padding oracle attack.

Now, let's explore a possible defense. Sending just a message encrypted with CBC mode guarantees us confidentiality if we use a secure block cipher, which we generally assume AES is. The lack of authentication and integrity checking mean that we as attackers are allowed to freely tamper with messages in order to decrypt them. It stands to reason that added some authentication and integrity checks would prevent us from successfully modifying messages for the server's examination. For simplicity, let's consider just two possible methods of achieving this: MAC-then-encrypt and encrypt-then-MAC, both with a secure MAC algorithm.

## 4.1

Does MAC-then-encrypt prevent users from exploiting our padding oracle attack? If yes, explain why. If not, explain how to modify the attack so that it still works.

**<span style = "color: red">(YOUR ANSWER HERE)</span>**

## 4.2

Does encrypt-then-MAC prevent users from exploiting our padding oracle attack? If yes, explain why. If not, explain how to modify the attack so that it still works.

**<span style = "color: red">(YOUR ANSWER HERE)</span>**

# Question 5: Mid-Semester Feedback
Crazy as it sounds, we are almost halfway through the class! The staff would appreciate your honest feedback so we can see what you all think we are doing well, and what we can improve on.

**Please fill out the anonymous mid-semester feedback form [here](https://forms.gle/ivrkFbUP2oKRePpD9). Upon completing it, enter the passphrase below**

**<span style = "color: red">(PASSPHRASE HERE)</span>**

# Question 6: Lab Feedback

This is the first time 161 has offered this lab, and we would greatly appreciate your feedback!

**Please fill out the anonymous lab feedback form [here](https://forms.gle/gpzcyzyuXq38un9w8). Upon completing it, enter the passphrase below**

**<span style = "color: red">(PASSPHRASE HERE)</span>**

# Question 7: SHA Length-Extension

In lecture, Nick let slip that SHA-2, great as (we currently think) it is, is vulnerable to an attack called a *length-extension attack*. In this question, we'll walk you through how the SHA-2 algorithm actually works and show you how an attacker could use that knowledge to forge SHA-2 hashes based on (message, hash) pairs that they observe.

## A Qualitative Explanation

SHA-2 (and its older, broken siblings MD5 and SHA-1) follow an algorithm that looks roughly like this:

```
Let B1, B2, ..., Bn = M split in
```